In [5]:
# import libraries

import pandas as pd
import os
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score

In [43]:
# import files

df_info = pd.read_csv("working_files/games_info_prep.csv")
df_scores = pd.read_csv("working_files/scores_prep.csv")

In [44]:
# merge files

df_scores_info = df_scores.merge(df_info, on = 'idgame')
display(df_scores_info)

,Unnamed: 0_x,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,...,capacity,people,line,over_under,attendance_info,dayofweek,dayofmonth,hour,month,minute
0,255,2018,1,Falcons,Eagles,12,18,401030710,6,1,...,69796,100,NaN,NaN,1,4,7,0,9,55
1,256,2018,1,Bengals,Colts,34,23,401030717,-11,0,...,63000,93,NaN,NaN,1,6,9,17,9,0
2,257,2018,1,Titans,Dolphins,20,27,401030716,7,1,...,65326,100,NaN,NaN,1,6,9,17,9,0
3,258,2018,1,49ers,Vikings,16,24,401030715,8,1,...,66468,100,NaN,NaN,1,6,9,17,9,0
4,259,2018,1,Texans,Patriots,20,27,401030714,7,1,...,65878,100,NaN,NaN,1,6,9,17,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,5349,2021,18,Jets,Bills,10,27,401326587,17,1,...,71621,90,Line: BUF -16.0,Over/Under: 43.0,1,6,9,21,1,25
1222,5351,2021,18,Patriots,Dolphins,24,33,401326592,9,1,...,65326,100,Line: NE -6.0,Over/Under: 41.0,1,6,9,21,1,25
1223,5352,2021,18,Seahawks,Cardinals,38,30,401326597,-8,0,...,65000,98,Line: ARI -5.5,Over/Under: 48.5,1,6,9,21,1,25
1224,5353,2021,18,Panthers,Buccaneers,17,41,401326596,24,1,...,65618,100,Line: TB -11.0,Over/Under: 43.0,1,6,9,21,1,25


In [50]:
display(df_scores["Unnamed: 0"])
df_scores_info["Unnamed: 0_x"]


0          0
1          1
2          2
3          3
4          4
        ... 
5350    5350
5351    5351
5352    5352
5353    5353
5354    5354
Name: Unnamed: 0, Length: 5355, dtype: int64

0        255
1        256
2        257
3        258
4        259
        ... 
1221    5349
1222    5351
1223    5352
1224    5353
1225    5354
Name: Unnamed: 0_x, Length: 1226, dtype: int64

In [13]:
# Basic stats
print("Number of rows : {}".format(df_scores_info.shape[0]))
print()

print("Display of dataset: ")
display(df_scores_info.head())
print()

print("Basics statistics: ")
data_desc = df_scores_info.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df_scores_info.isnull().sum()/df_scores_info.shape[0])


Number of rows : 1226

Display of dataset: 


,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,winner_away,...,capacity,people,line,over_under,attendance_info,dayofweek,dayofmonth,hour,month,minute
0,2018,1,Falcons,Eagles,12,18,401030710,6,1,0,...,69796,100,NaN,NaN,1,4,7,0,9,55
1,2018,1,Bengals,Colts,34,23,401030717,-11,0,1,...,63000,93,NaN,NaN,1,6,9,17,9,0
2,2018,1,Titans,Dolphins,20,27,401030716,7,1,0,...,65326,100,NaN,NaN,1,6,9,17,9,0
3,2018,1,49ers,Vikings,16,24,401030715,8,1,0,...,66468,100,NaN,NaN,1,6,9,17,9,0
4,2018,1,Texans,Patriots,20,27,401030714,7,1,0,...,65878,100,NaN,NaN,1,6,9,17,9,0



Basics statistics: 


,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,winner_away,...,capacity,people,line,over_under,attendance_info,dayofweek,dayofmonth,hour,month,minute
count,1226.000000,1226.000000,1226,1226,1226.000000,1226.00000,1.226000e+03,1226.000000,1226.000000,1226.000000,...,1226.000000,1226.000000,491,439,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000
unique,NaN,NaN,32,32,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,322,42,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Rams,Bengals,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Line: BAL -3.0,Over/Under: 44.5,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,41,41,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5,23,NaN,NaN,NaN,NaN,NaN,NaN
mean,2019.008157,9.190865,NaN,NaN,22.546493,23.73491,4.011323e+08,1.188418,0.538336,0.461664,...,71276.068515,77.854812,NaN,NaN,0.891517,5.095432,16.104405,14.993475,10.262643,8.821370
std,1.418800,5.064268,NaN,NaN,10.145306,10.36788,1.334634e+05,14.775752,0.498732,0.498732,...,8101.037588,35.773484,NaN,NaN,0.311116,1.919999,8.780527,7.239796,2.114175,11.214855
min,2017.000000,1.000000,NaN,NaN,0.000000,0.00000,4.009502e+08,-49.000000,0.000000,0.000000,...,55997.000000,0.000000,NaN,NaN,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,2018.000000,5.000000,NaN,NaN,16.000000,17.00000,4.010308e+08,-7.000000,0.000000,0.000000,...,65618.000000,83.000000,NaN,NaN,1.000000,6.000000,9.000000,17.000000,10.000000,0.000000
50%,2019.000000,9.000000,NaN,NaN,23.000000,24.00000,4.011280e+08,2.000000,1.000000,0.000000,...,68740.000000,95.000000,NaN,NaN,1.000000,6.000000,16.000000,18.000000,11.000000,0.000000
75%,2020.000000,14.000000,NaN,NaN,30.000000,31.00000,4.012203e+08,10.000000,1.000000,1.000000,...,75000.000000,100.000000,NaN,NaN,1.000000,6.000000,24.000000,18.000000,12.000000,20.000000



Percentage of missing values: 


season              0.000000
week                0.000000
awayteam            0.000000
hometeam            0.000000
awayscore           0.000000
homescore           0.000000
idgame              0.000000
score_abs           0.000000
winner_home         0.000000
winner_away         0.000000
date                0.000000
stade               0.000000
location            0.000000
attendance          0.000000
capacity            0.000000
people              0.000000
line               59.951060
over_under         64.192496
attendance_info     0.000000
dayofweek           0.000000
dayofmonth          0.000000
hour                0.000000
month               0.000000
minute              0.000000
dtype: float64

In [6]:
df_scores_info.columns

Index(['Unnamed: 0_x', 'season', 'week', 'awayteam', 'hometeam', 'awayscore',
       'homescore', 'idgame', 'score_abs', 'winner_home', 'winner_away',
       'Unnamed: 0_y', 'date', 'stade', 'location', 'attendance', 'capacity',
       'people', 'line', 'over_under', 'attendance_info', 'dayofweek',
       'dayofmonth', 'hour', 'month', 'minute'],
      dtype='object')

In [10]:
df_scores_info.drop(columns=['Unnamed: 0_x','Unnamed: 0_y' ], inplace = True)

In [11]:
df_scores_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1226 entries, 0 to 1225
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   season           1226 non-null   int64 
 1   week             1226 non-null   int64 
 2   awayteam         1226 non-null   object
 3   hometeam         1226 non-null   object
 4   awayscore        1226 non-null   int64 
 5   homescore        1226 non-null   int64 
 6   idgame           1226 non-null   int64 
 7   score_abs        1226 non-null   int64 
 8   winner_home      1226 non-null   int64 
 9   winner_away      1226 non-null   int64 
 10  date             1226 non-null   object
 11  stade            1226 non-null   object
 12  location         1226 non-null   object
 13  attendance       1226 non-null   int64 
 14  capacity         1226 non-null   int64 
 15  people           1226 non-null   int64 
 16  line             491 non-null    object
 17  over_under       439 non-null    

In [16]:
# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [17]:
features_columns = ["season","week", "awayteam", "hometeam",'stade','attendance','capacity','people','attendance_info','dayofweek',
                    'dayofmonth','hour','month', 'minute']
X = df_scores_info.loc[:,features_columns]
y = df_scores_info.loc[:,"winner_home"]
print(X.head())
print(y.head())

   season  week awayteam  hometeam                    stade  attendance  \
0    2018     1  Falcons    Eagles  Lincoln Financial Field       69696   
1    2018     1  Bengals     Colts        Lucas Oil Stadium       58699   
2    2018     1   Titans  Dolphins        Hard Rock Stadium       65184   
3    2018     1    49ers   Vikings        U.S. Bank Stadium       66673   
4    2018     1   Texans  Patriots         Gillette Stadium       65878   

   capacity  people  attendance_info  dayofweek  dayofmonth  hour  month  \
0     69796     100                1          4           7     0      9   
1     63000      93                1          6           9    17      9   
2     65326     100                1          6           9    17      9   
3     66468     100                1          6           9    17      9   
4     65878     100                1          6           9    17      9   

   minute  
0      55  
1       0  
2       0  
3       0  
4       0  
0    1
1    0
2    1

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.2,
                                                    stratify = y , ## Statify splitting when you're training a classification model !
                                                    random_state = 42)

In [26]:
one = OneHotEncoder()
sc = StandardScaler()
classifier = LogisticRegression()

In [28]:
# Create pipeline for numeric features
numeric_features = ["season","week", 'attendance','capacity','attendance_info', 'people', 'dayofweek',
                    'dayofmonth','hour','month', 'minute'] # numeric columns in X_train/X_test
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = ["awayteam", "hometeam",'stade'] # categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop="first")) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTranformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocessings on train set
X_train = preprocessor.fit_transform(X_train)

# Preprocessings on test set
X_test = preprocessor.transform(X_test) # Don't fit again !!

In [29]:
classifier.fit(X_train, y_train)
scores = cross_val_score(classifier, X_train, y_train)
print("Cross-validated score : {}\nStd : {}".format(scores.mean(), scores.std()))

Cross-validated score : 0.6010204081632653
Std : 0.03809827490325398


In [40]:
round(classifier.score(X_test, y_test),4)

0.5894

In [41]:
from sklearn.svm import SVC
clf_svm = SVC(kernel='rbf')
clf_svm = clf_svm.fit(X_train, y_train)
scores = cross_val_score(clf_svm, X_train, y_train)
print("Cross-validated score : {}\nStd : {}".format(scores.mean(), scores.std()))

Cross-validated score : 0.5469387755102041
Std : 0.017196224026890545


In [42]:
round(clf_svm.score(X_test, y_test),4)

0.6057